In [2]:
!git clone https://github.com/pdan101/catanatron.git
!pip install -r catanatron/dev-requirements.txt
!pip install -e catanatron/catanatron_core
!pip install -e catanatron/catanatron_server
!pip install -e catanatron/catanatron_gym
!pip install -e catanatron/catanatron_experimental
exit() # Forcefully restart runtime to picks up installed requirements

fatal: destination path 'catanatron' already exists and is not an empty directory.
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached attrs-21.4.0-py2.py3-none-any.whl (60 kB)
  Using cached black-22.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
  Using cached cachetools-5.0.0-py3-none-any.whl (9.1 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached charset_normalizer-2.0.11-py3-none-any.whl (39 kB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached coverage-6.3.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (210 kB)
  Using cached coveralls-3.3.1-py2.py3-none-any.whl (14 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cac

In [7]:
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000, maxWidth:10000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

<IPython.core.display.Javascript object>

In [1]:
from catanatron import Game, RandomPlayer, Color

from catanatron_experimental.machine_learning.players.minimax import AlphaBetaPlayer
from catanatron_experimental.play import play_batch

from catanatron_experimental.machine_learning.players.value import DEFAULT_WEIGHTS


In [2]:
test_weights = DEFAULT_WEIGHTS.copy()
for k, v in test_weights.items():
    test_weights[k] = 0
test_weights['enemy_production'] = -5e8

In [8]:
players = [
    AlphaBetaPlayer(Color.RED, params=test_weights, value_fn_builder_name="C"),
    AlphaBetaPlayer(Color.BLUE)
]

wins, results_by_player, games = play_batch(2, players)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Output()

                    Last 2 Games                    
    ╷          ╷       ╷        ╷         ╷         
  # │ SEATING  │ TURNS │ RED VP │ BLUE VP │ WINNER  
╶───┼──────────┼───────┼────────┼─────────┼────────╴
  1 │ RED,BLUE │    89 │      4 │      10 │ BLUE    
  2 │ RED,BLUE │    61 │      3 │      10 │ BLUE    
    ╵          ╵       ╵        ╵         ╵

                                                  Player Summary                                                   
                                                                    ╷      ╷      ╷     ╷      ╷     ╷      ╷      
                                                                    │      │      │     │      │     │      │ AVG  
                                                                    │      │  AVG │ AVG │  AVG │ AVG │  AVG │ DEV  
                                                                    │ WINS │   VP │ SE… │ CIT… │ RO… │ ARMY │  VP  
╶───────────────────────────────────────────────────────────────────┼──────┼──────┼─────┼──────┼─────┼──────┼─────╴
  AlphaBetaPlayer:RED(depth=2,value_fn=contender_fn,prunning=False) │    0 │ 3.50 │ 2.… │ 0.00 │ 0.… │ 0.00 │ 0.…  
  AlphaBetaPlayer:BLUE(depth=2,value_fn=base_fn,prunning=False)     │    2 │ 10.… │ 4.… │ 2.50 │ 0.… │ 0.00 │ 0.…  
                                                                    ╵      ╵      ╵     ╵      ╵     ╵      ╵

              Game Summary              
            ╷           ╷               
  AVG TICKS │ AVG TURNS │ AVG DURATION  
╶───────────┼───────────┼──────────────╴
     208.50 │     75.00 │   7.317 secs  
            ╵           ╵